In [1]:
import os
import warnings
import uuid
from dotenv import load_dotenv

from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings
from pydantic import BaseModel, Field
from IPython.display import display, Image
from langchain.schema import Document
from langchain import hub

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig, RunnablePassthrough, RunnableLambda
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
load_dotenv()

os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"
# os.environ["HF_HOME"] = "./cache/"

In [3]:
from langchain_teddynote.tools import TavilySearch

tools = [TavilySearch(max_results=3)]

In [6]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer in Korean."
        ),
        (
            "human",
            "{messages}"
        )
    ]
)

llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

# ReAct 에이전트 생성
agent_executor = create_react_agent(llm, tools, prompt=prompt)

In [7]:
agent_executor.invoke(
    {"messages":[("user", "랭체인 한국어 튜토리얼에 대해서 설명해줘")]}
)

{'messages': [HumanMessage(content='랭체인 한국어 튜토리얼에 대해서 설명해줘', additional_kwargs={}, response_metadata={}, id='e5af4857-e29f-4744-95d5-02b7fbd83f85'),
  AIMessage(content='좋아요. 랭체인(LangChain) 한국어 튜토리얼에 대해 개괄적으로 설명해 드릴게요. 아래 내용은 한국어로 LLM 기반 애플리케이션을 처음부터 차근차근 만들어 보는 것을 목표로 한 초안 구성입니다. 필요에 따라 깊이와 속도를 조정해 드릴 수 있어요.\n\n1) 랭체인이란?\n- 랭체인은 LLM(대형 언어 모델)을 활용한 애플리케이션 개발을 쉽게 해주는 파이썬 라이브러리예요.\n- 주요 구성 요소: 프롬프트 관리(prompts), 체인(chains), 에이전트(agents), 도구(tools), 메모리(memory), 벡터 저장소(vector stores) 등.\n- 한국어 튜토리얼의 포커스: 한국어 데이터로도 프롬프트를 다루고, 한국어 환경에서의 디버깅·실전 운용 방법을 중점적으로 다루는 것.\n\n2) 한국어 튜토리얼의 목표\n- 한국어로 LLM 애플리케이션을 설계하고 구현하는 기본 역량 확보\n- 프롬프트 템플릿 작성과 체인 구성의 흐름 이해\n- 메모리와 에이전트를 활용한 대화 흐름 관리\n- 벡터 저장소를 이용한 지식 기반 질의응답 구현\n- 한국어 데이터의 특성(토큰화, 형태소, 다의어 처리 등)에 대한 실무 팁 습득\n- 배포 전 점검 항목과 비용 관리의 기본 이해\n\n3) 추천 학습 순서\n- 환경 구성과 기초 다지기\n  - 파이썬 설치, 가상환경 만들기, langchain 및 필요한 패키지 설치\n  - 기본적인 텍스트 데이터 처리 방법 복습(한국어 토큰화 예시)\n- 프롬프트 템플릿 이해\n  - PromptTemplate의 역할, 프롬프트 파라미터 만들기\n  - 한국어 프롬프트 설계 시 주의점(공백, 한글 인코딩, 톤)\n- LLMCha

In [ ]:
# 1단계 : 문서 로드
loader = PyMuPDFLoader("SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 2단계 : 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 3단계 : 임베딩
hf_embeddings = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-m3",
    model_kwargs = {"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

# 4단계 : 벡터스토어 저장/불러오기
try:
    vectorstore = FAISS.load_local(
        folder_path="faiss_db",
        index_name="faiss_index",
        embeddings=hf_embeddings,
        allow_dangerous_deserialization=True,
    )
except:
    vectorstore = FAISS.from_documents(split_documents, hf_embeddings)
    vectorstore.save_local("faiss_db", "faiss_index")

# vectorstore.add_documents(new_split_documents)
# vectorstore.save_local("faiss_db", "faiss_index")

# 5단계 : 검색기 Retriever 생성
retriever = vectorstore.as_retriever()

# 6단계 : 프롬프트
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
You must include `page` number in your answer.
Answer in Korean.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

# 7단계 : LLM 생성
# llm = ChatOpenAI(model_name="gpt-5-nano", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))
llm = ChatOllama(model="llama3.2:4b", temperature=0, base_url="http://localhost:11434")

# 8단계 : chain 생성
def format_docs(docs):
    return "\n\n".join(
        f"[page {d.metadata.get('page', 0) + 1}] {d.page_content}" for d in docs
    )

chain = (
    {"context":retriever | RunnableLambda(format_docs), "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)